
# ViEWS 3 constituent models 
## ViEWS production system, cm level


This notebook trains a set of regression models for use in the monthly updated ViEWS predicting fatalities ensemble

The notebook does the following: 
1. Retrieves data through querysets and stores in DataSets, a list of dictionaries
2. Specifies the metadata of a number of models, stores in ModelList, a list of dictionaries
3. Trains the models in ModelList, stores the trained objects in model storage and prediction storage
4. Saves part of ModelList as csv and the rest as pickles


## Importing modules

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
# sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression

from xgboost import XGBRegressor
from xgboost import XGBClassifier
from xgboost import XGBRFRegressor, XGBRFClassifier

from lightgbm import LGBMClassifier, LGBMRegressor

# Views 3
from viewser.operations import fetch
import views_runs
from views_partitioning import data_partitioner, legacy
from stepshift import views
import views_dataviz
from views_runs import storage
from views_runs.storage import store, retrieve, fetch_metadata

from views_forecasts.extensions import *

# Other packages
import pickle as pkl

# Packages from viewsforecasting repository

#from Ensembling import CalibratePredictions, RetrieveStoredPredictions, mean_sd_calibrated, gam_calibrated
import os
import sys
sys.path.append('../')
sys.path.append('../Tools')
sys.path.append('../Intermediates')
from FetchData import FetchData, RetrieveFromList, document_queryset, ReturnQsList, document_ensemble
from ViewsEstimators import *


## Common parameters

In [3]:
#!conda list | grep views

In [4]:
# To do:
# find out why and where missingness occurs

In [5]:
# Common parameters:
dev_id = 'Fatalities002'
run_id = dev_id

# Generating a new run if necessary

#try:
#    ViewsMetadata().new_run(name=run_id,description='Developing the fatalities model for FCDO',min_month=1,max_month=999)
#except KeyError:
#    if 'devel' not in run_id:
#        warnings.warn('You are overwriting a production system')

RerunQuerysets = True

FutureStart = 508
steps = [*range(1, 36+1, 1)] # Which steps to train and predict for
fi_steps = [1,3,6,12,36] # Which steps to present feature importances for
#steps = [1,3,6,12,36]
#fi_steps = [1,3,6,12,36]

# Specifying partitions
calib_partitioner_dict = {"train":(121,396),"predict":(397,444)}
test_partitioner_dict = {"train":(121,444),"predict":(445,492)}
future_partitioner_dict = {"train":(121,492),"predict":(493,504)}
calib_partitioner =  views_runs.DataPartitioner({"calib":calib_partitioner_dict})
test_partitioner =  views_runs.DataPartitioner({"test":test_partitioner_dict})
future_partitioner =  views_runs.DataPartitioner({"future":future_partitioner_dict})

Mydropbox = f'/Users/{os.getlogin()}/Dropbox (ViEWS)/ViEWS'
print('Setting Mydropbox to',Mydropbox)

Setting Mydropbox to /Users/havardhegre1/Dropbox (ViEWS)/ViEWS


# Retrieve data

In [6]:
# Create Markdown documentation of all querysets used
level = 'cm'
qslist = ReturnQsList(level)
document_queryset(qslist,dev_id)

 .    fatalities002_baseline; A dataset with 6 columns, with data between t 1 and 852. (213 units)
 .    fatalities002_topics_stub; A dataset with 62 columns, with data between t 1 and 852. (213 units)
 .    fatalities002_aquastat_stub; A dataset with 11 columns, with data between t 1 and 852. (213 units)
 .    fatalities002_cm_conflict_history_stub; A dataset with 24 columns, with data between t 1 and 852. (213 units)
 .    fatalities002_cm_conflict_history_ext; A dataset with 33 columns, with data between t = 1 and 852. (213 units)
 .    fatalities002_vdem_short_stub; A dataset with 57 columns, with data between t 1 and 852. (213 units)
 .    fatalities002_wdi_short_stub; A dataset with 26 columns, with data between t 1 and 852. (213 units)
 .    fatalities002_joint_narrow; A dataset with 39 columns, with data between t 1 and 852. (213 units)
 .    fatalities002_joint_broad_stub; A dataset with 102 columns, with data between t 1 and 852. (213 units)
 .    fatalities002_faostat_stub;A

In [7]:
from FetchData import fetch_cm_data_from_model_def

Datasets=fetch_cm_data_from_model_def(qslist)

 .    topics_002: A dataset with 68 columns, with data between t = 1 and 852; 213 units.
 .    conflict_ln: A dataset with 30 columns, with data between t = 1 and 852; 213 units.
 .    vdem_short: A dataset with 63 columns, with data between t = 1 and 852; 213 units.
 .    aquastat: A dataset with 17 columns, with data between t = 1 and 852; 213 units.
 .    conflictlong_ln: A dataset with 63 columns, with data between t = 1 and 852; 213 units.
 .    baseline002: A dataset with 6 columns, with data between t = 1 and 852; 213 units.
 .    all_features: A dataset with 186 columns, with data between t = 1 and 852; 213 units.
 .    faostat: A dataset with 41 columns, with data between t = 1 and 852; 213 units.
 .    faoprices: A dataset with 17 columns, with data between t = 1 and 852; 213 units.
 .    imfweo: A dataset with 11 columns, with data between t = 1 and 852; 213 units.
 .    wdi_short: A dataset with 32 columns, with data between t = 1 and 852; 213 units.
 .    joint_broad: A da

# Generating predictions
Using the ViEWS3 partitioning/stepshifting syntax. Training models for A: calibration partition and B: test partition, to test out some calibration routines. Most models trained with ln_ged_sb_best as outcome.

In [8]:
dev_id

'Fatalities002'

In [9]:
Datasets[0]['df'].head()

ln_ged_sb_dep  ln_ged_sb  wdi_sp_pop_totl  \
month_id country_id                                              
1        1                     0.0        0.0         780153.0   
         2                     0.0        0.0         359531.0   
         3                     0.0        0.0        1084744.0   
         4                     0.0        0.0       15182611.0   
         5                     0.0        0.0         155525.0   

                     topic0_religion_t1  topic0_religion_t13  \
month_id country_id                                            
1        1                      0.88053              0.88053   
         2                      0.00000              0.00000   
         3                      0.85479              0.85479   
         4                      0.05139              0.05139   
         5                      0.00000              0.00000   

                     topic1_politics_t1  topic1_politics_t13  \
month_id country_id                                            
1        1                     0.541010             0.541010   
         2                     0.000000             0.000000   
         3                    32.893841            32.893841   
         4                    17.967770            17.967770   
         5                     0.000000             0.000000   

                     topic2_sanctions_t1  topic2_sanctions_t13  \
month_id country_id                                              
1        1                       0.26324               0.26324   
         2                       0.00000               0.00000   
         3                       0.85479               0.85479   
         4                       0.39640               0.39640   
         5                       0.00000               0.00000   

                     topic3_life_t1  ...  splag_topic5_media_t1_stock  \
month_id country_id                  ...                                
1        1                 14.33693  ...                      8.86444   
         2                  0.00000  ...                      4.70923   
         3                  9.31762  ...                      0.00000   
         4                  5.85506  ...                      5.14827   
         5                  0.00000  ...                      0.00000   

                     splag_topic6_economics_t1_stock  \
month_id country_id                                    
1        1                                 48.831049   
         2                                 21.028550   
         3                                  0.000000   
         4                                 34.749929   
         5                                  0.000000   

                     splag_topic7_health_t1_stock  \
month_id country_id                                 
1        1                                3.92519   
         2                                6.43111   
         3                                0.00000   
         4                                8.96491   
         5                                0.00000   

                     splag_topic8_china_t1_stock  \
month_id country_id                                
1        1                               0.47482   
         2                               0.47872   
         3                               0.00000   
         4                               0.92614   
         5                               0.00000   

                     splag_topic9_foreign_t1_stock  \
month_id country_id                                  
1        1                               10.191550   
         2                               11.422680   
         3                                0.000000   
         4                               19.777639   
         5                                0.000000   

                     splag_topic10_conflict_t1_stock  \
month_id country_id                                    
1        1                                   6.36659   
        

In [10]:
from views_runs import ModelMetadata 
help(ModelMetadata)

Help on class ModelMetadata in module views_schema.models:

class ModelMetadata(pydantic.main.BaseModel)
 |  ModelMetadata(*, author: str, queryset_name: str, train_start: int, train_end: int, steps: List[int] = None, training_date: datetime.datetime) -> None
 |  
 |  ModelMetadata
 |  =============
 |  
 |  Data used to organize model objects.
 |  
 |  parameters:
 |      author (str): Name of the user that authored the model object.
 |      queryset_name (str): Name of the queryset used to train the model
 |      train_start (int): Month identifier for training start date
 |      train_start (int): Month identifier for training end date
 |      training_date (datetime.datetime): Timestamp for training date (use datetime.datetime.now())
 |  
 |  example:
 |  
 |      # Instantiate the class with values
 |  
 |      my_metadata = ModelMetadata(
 |          author = "my_name",
 |          queryset_name = "my_queryset",
 |          train_start = 1,
 |          train_end = 300,
 |        

## Checking missingness and infinity values

In [11]:
N=51
for i in range(len(Datasets)):
    df = Datasets[i]['df']
    print(Datasets[i]['Name'])
    for col in df.iloc[: , :N].columns:
        print(col,len(df[col]), 'missing:', df[col].isnull().sum(), 'infinity:', np.isinf(df).values.sum())


topics_002
ln_ged_sb_dep 158230 missing: 0 infinity: 0
ln_ged_sb 158230 missing: 0 infinity: 0
wdi_sp_pop_totl 158230 missing: 11 infinity: 0
topic0_religion_t1 158230 missing: 5 infinity: 0
topic0_religion_t13 158230 missing: 11 infinity: 0
topic1_politics_t1 158230 missing: 5 infinity: 0
topic1_politics_t13 158230 missing: 11 infinity: 0
topic2_sanctions_t1 158230 missing: 5 infinity: 0
topic2_sanctions_t13 158230 missing: 11 infinity: 0
topic3_life_t1 158230 missing: 5 infinity: 0
topic3_life_t13 158230 missing: 11 infinity: 0
topic4_energy_t1 158230 missing: 5 infinity: 0
topic4_energy_t13 158230 missing: 11 infinity: 0
topic5_media_t1 158230 missing: 5 infinity: 0
topic5_media_t13 158230 missing: 11 infinity: 0
topic6_economics_t1 158230 missing: 5 infinity: 0
topic6_economics_t13 158230 missing: 11 infinity: 0
topic7_health_t1 158230 missing: 5 infinity: 0
topic7_health_t13 158230 missing: 11 infinity: 0
topic8_china_t1 158230 missing: 5 infinity: 0
topic8_china_t13 158230 missin

wdi_sp_pop_totl 158230 missing: 11 infinity: 0
ln_ged_sb_tlag_1 158230 missing: 5 infinity: 0
ln_ged_sb_tlag_2 158230 missing: 5 infinity: 0
ln_ged_sb_tlag_3 158230 missing: 5 infinity: 0
ln_ged_sb_tlag_4 158230 missing: 5 infinity: 0
ln_ged_sb_tlag_5 158230 missing: 5 infinity: 0
ln_ged_sb_tlag_6 158230 missing: 11 infinity: 0
ln_ged_sb_tsum_24 158230 missing: 0 infinity: 0
ln_ged_os_tlag_1 158230 missing: 5 infinity: 0
ln_ged_sb_tsum_12 158230 missing: 0 infinity: 0
ln_ged_sb_tsum_48 158230 missing: 0 infinity: 0
ln_ged_ns_tlag_1 158230 missing: 5 infinity: 0
ln_ged_ns_tlag_2 158230 missing: 5 infinity: 0
ln_ged_os_tlag_2 158230 missing: 5 infinity: 0
ln_acled_sb_tlag_1 158230 missing: 1437 infinity: 0
ln_acled_sb_tlag_2 158230 missing: 1437 infinity: 0
ln_acled_os_tlag_1 158230 missing: 1437 infinity: 0
ln_acled_os_tlag_2 158230 missing: 1437 infinity: 0
ln_acled_ns_tlag_1 158230 missing: 1437 infinity: 0
ln_acled_ns_tlag_2 158230 missing: 1437 infinity: 0
decay_ged_sb_5 158230 miss

wdi_ms_mil_xpnd_zs 158230 missing: 11 infinity: 0
wdi_nv_agr_totl_kd 158230 missing: 11 infinity: 0
wdi_nv_agr_totl_kn 158230 missing: 11 infinity: 0
wdi_ny_gdp_mktp_kd 158230 missing: 11 infinity: 0
wdi_se_enr_prim_fm_zs 158230 missing: 11 infinity: 0
wdi_se_enr_prsc_fm_zs 158230 missing: 11 infinity: 0
wdi_sh_sta_maln_zs 158230 missing: 11 infinity: 0
wdi_sh_sta_stnt_zs 158230 missing: 11 infinity: 0
wdi_sl_tlf_totl_fe_zs 158230 missing: 11 infinity: 0
wdi_sm_pop_refg_or 158230 missing: 11 infinity: 0
wdi_sm_pop_netm 158230 missing: 11 infinity: 0
wdi_sm_pop_totl_zs 158230 missing: 11 infinity: 0
wdi_sp_dyn_imrt_in 158230 missing: 11 infinity: 0
wdi_sh_dyn_mort_fe 158230 missing: 11 infinity: 0
wdi_sp_pop_14_fe_zs 158230 missing: 11 infinity: 0
wdi_sp_pop_1564_fe_zs 158230 missing: 11 infinity: 0
wdi_sp_pop_65up_fe_zs 158230 missing: 11 infinity: 0
wdi_sp_pop_grow 158230 missing: 11 infinity: 0
wdi_sp_urb_totl_in_zs 158230 missing: 11 infinity: 0
splag_wdi_sl_tlf_totl_fe_zs 158230 mi

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# Specify models in ensemble

In [12]:
from ModelDefinitions import DefineEnsembleModels

ModelList = DefineEnsembleModels('cm')
    
for imodel,model in enumerate(ModelList):
    print(imodel, model['modelname'], model['data_train'])

0 fatalities002_baseline_rf baseline002
1 fatalities002_conflicthistory_rf conflict_ln
2 fatalities002_conflicthistory_gbm conflict_ln
3 fatalities002_conflicthistory_hurdle_lgb conflict_ln
4 fatalities002_conflicthistory_long_xgb conflictlong_ln
5 fatalities002_vdem_hurdle_xgb vdem_short
6 fatalities002_wdi_rf wdi_short
7 fatalities002_topics_rf topics_002
8 fatalities002_topics_xgb topics_002
9 fatalities002_topics_hurdle_lgb topics_002
10 fatalities002_joint_broad_rf joint_broad
11 fatalities002_joint_broad_hurdle_rf joint_broad
12 fatalities002_joint_narrow_xgb joint_narrow
13 fatalities002_joint_narrow_hurdle_xgb joint_narrow
14 fatalities002_joint_narrow_hurdle_lgb joint_narrow
15 fatalities002_all_pca3_xgb all_features
16 fatalities002_aquastat_rf aquastat
17 fatalities002_faostat_rf faostat
18 fatalities002_faoprices_rf faoprices
19 fatalities002_imfweo_rf imfweo
20 fat_hh20_Markov_glm joint_narrow
21 fat_hh20_Markov_rf joint_narrow


In [13]:
ModelList

[{'modelname': 'fatalities002_baseline_rf',
  'algorithm': XGBRFRegressor(base_score=None, booster=None, callbacks=None,
                 colsample_bylevel=None, colsample_bytree=None,
                 early_stopping_rounds=None, enable_categorical=False,
                 eval_metric=None, gamma=None, gpu_id=None, grow_policy=None,
                 importance_type=None, interaction_constraints=None, max_bin=None,
                 max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
                 max_leaves=None, min_child_weight=None, missing=nan,
                 monotone_constraints=None, n_estimators=300, n_jobs=12,
                 num_parallel_tree=None, objective='reg:squarederror',
                 predictor=None, random_state=None, reg_alpha=None,
                 sampling_method=None, scale_pos_weight=None, ...),
  'depvar': 'ln_ged_sb_dep',
  'data_train': 'baseline002',
  'queryset': 'fatalities002_baseline',
  'preprocessing': 'float_it',
  'level': 'cm',
  'desc

In [14]:
document_ensemble(ModelList,'sb')

0 fatalities002_baseline_rf baseline002
1 fatalities002_conflicthistory_rf conflict_ln
2 fatalities002_conflicthistory_gbm conflict_ln
3 fatalities002_conflicthistory_hurdle_lgb conflict_ln
4 fatalities002_conflicthistory_long_xgb conflictlong_ln
5 fatalities002_vdem_hurdle_xgb vdem_short
6 fatalities002_wdi_rf wdi_short
7 fatalities002_topics_rf topics_002
8 fatalities002_topics_xgb topics_002
9 fatalities002_topics_hurdle_lgb topics_002
10 fatalities002_joint_broad_rf joint_broad
11 fatalities002_joint_broad_hurdle_rf joint_broad
12 fatalities002_joint_narrow_xgb joint_narrow
13 fatalities002_joint_narrow_hurdle_xgb joint_narrow
14 fatalities002_joint_narrow_hurdle_lgb joint_narrow
15 fatalities002_all_pca3_xgb all_features
16 fatalities002_aquastat_rf aquastat
17 fatalities002_faostat_rf faostat
18 fatalities002_faoprices_rf faoprices
19 fatalities002_imfweo_rf imfweo
20 fat_hh20_Markov_glm joint_narrow
21 fat_hh20_Markov_rf joint_narrow


In [15]:
# Loop that checks whether the model exists, retrains if not, 
# and stores the predictions if they have not been stored before for this run.
# To do: set the data_preprocessing to the function in the model dictionary

level = 'cm'
includeFuture = False

from views_runs import Storage, StepshiftedModels
from views_partitioning.data_partitioner import DataPartitioner
from viewser import Queryset, Column
from views_runs import operations
from views_runs.run_result import RunResult

i = 0
for model in ModelList:
    if model['algorithm'] != 'Rscript':
        force_retrain = False
        modelstore = storage.Storage()
        ct = datetime.now()
        print(i, model['modelname'])
        print('Calibration partition', ct)
        model['Algorithm_text'] = str(model['algorithm'])
        model['RunResult_calib'] = RunResult.retrain_or_retrieve(
                retrain            = force_retrain,
                store              = modelstore,
                partitioner        = DataPartitioner({"calib":calib_partitioner_dict}),
                stepshifted_models = StepshiftedModels(model['algorithm'], steps, model['depvar']),
                dataset            = RetrieveFromList(Datasets,model['data_train']),
                queryset_name      = model['queryset'],
                partition_name     = "calib",
                timespan_name      = "train",
                storage_name       = model['modelname'] + '_calib',
                author_name        = "HH",
        )

    #    model['predstore_calib'] = level +  '_' + model['modelname'] + '_calib'
        ct = datetime.now()
        print('Trying to retrieve predictions', ct)
        try:
            predictions_calib = pd.DataFrame.forecasts.read_store(run=run_id, name=model['predstore_calib'])
        except KeyError:
            print(model['predstore_calib'], ', run',  run_id, 'does not exist, predicting')
            predictions_calib = model['RunResult_calib'].run.predict("calib","predict", model['RunResult_calib'].data)
            predictions_calib.forecasts.set_run(run_id)
            predictions_calib.forecasts.to_store(name=model['predstore_calib'])

        ct = datetime.now()
        print('Test partition', ct)
        modelstore = storage.Storage()
        model['RunResult_test'] = RunResult.retrain_or_retrieve(
                retrain            = force_retrain,
                store              = modelstore,
                partitioner        = DataPartitioner({"test":test_partitioner_dict}),
                stepshifted_models = StepshiftedModels(model['algorithm'], steps, model['depvar']),
                dataset            = RetrieveFromList(Datasets,model['data_train']),
                queryset_name      = model['queryset'],
                partition_name     = "test",
                timespan_name      = "train",
                storage_name       = model['modelname'] + '_test',
                author_name        = "HH",
        )
        ct = datetime.now()
        print('Trying to retrieve predictions', ct)
    #    model['predstore_test'] = level +  '_' + model['modelname'] + '_test'
        try:
            predictions_test = pd.DataFrame.forecasts.read_store(run=run_id, name=model['predstore_test'])
        except KeyError:
            print(model['predstore_test'], ', run', run_id, 'does not exist, predicting')
            predictions_test = model['RunResult_test'].run.predict("test","predict",model['RunResult_test'].data)
            predictions_test.forecasts.set_run(run_id)
            predictions_test.forecasts.to_store(name=model['predstore_test'])
        # Predictions for true future
        if includeFuture:
            ct = datetime.now()
            print('Future', ct)
            modelstore = storage.Storage()
            model['RunResult_future'] = RunResult.retrain_or_retrieve(
                    retrain            = force_retrain,
                    store              = modelstore,
                    partitioner        = DataPartitioner({"test":future_partitioner_dict}),
                    stepshifted_models = StepshiftedModels(model['algorithm'], steps, model['depvar']),
                    dataset            = RetrieveFromList(Datasets,model['data_train']),
                    queryset_name      = model['queryset'],
                    partition_name     = "test",
                    timespan_name      = "train",
                    storage_name       = model['modelname'] + '_future',
                    author_name        = "HH",
            )
            ct = datetime.now()
            print('Trying to retrieve predictions', ct)
            model['predstore_future'] = level +  '_' + model['modelname'] + '_f' + str(FutureStart)
            try:
                predictions_future = pd.DataFrame.forecasts.read_store(run=run_id, name=model['predstore_future'])
            except KeyError:
                print(model['predstore_future'], ', run', run_id, 'does not exist, predicting')
                predictions_future = model['RunResult_future'].run.future_point_predict(FutureStart,model['RunResult_future'].data)
                predictions_future.forecasts.set_run(run_id)
                predictions_future.forecasts.to_store(name=model['predstore_future'])  
        print('**************************************************************')
    i = i + 1

print('All done')

0 fatalities002_baseline_rf
Calibration partition 2022-09-21 07:44:43.702963
 * == Performing a run: "fatalities002_baseline_rf_calib" == * 
Model object named "fatalities002_baseline_rf_calib" with equivalent metadata already exists.
Fetching "fatalities002_baseline_rf_calib" from storage
[07:44:54] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1660208952489/work/src/learner.cc:1040: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[07:44:54] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1660208952489/work/src/learner.cc:749: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for 


[07:44:54] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1660208952489/work/src/learner.cc:749: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.
[07:44:54] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1660208952489/work/src/learner.cc:438: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[07:44:54] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1660208952489/work/src/learner.cc:1040: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model b


[07:44:54] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1660208952489/work/src/learner.cc:1040: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[07:44:54] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1660208952489/work/src/learner.cc:749: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.
[07:44:54] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1660208952489/work/src/learner.cc:438: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model b

[07:44:54] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1660208952489/work/src/learner.cc:749: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.
[07:44:54] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1660208952489/work/src/learner.cc:438: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[07:44:54] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1660208952489/work/src/learner.cc:1040: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by


[07:44:54] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1660208952489/work/src/learner.cc:1040: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[07:44:54] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1660208952489/work/src/learner.cc:749: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.
[07:44:54] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1660208952489/work/src/learner.cc:438: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model b

[07:44:54] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1660208952489/work/src/learner.cc:749: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.
[07:44:54] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1660208952489/work/src/learner.cc:438: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[07:44:55] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1660208952489/work/src/learner.cc:1040: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by

AttributeError: 'XGBModel' object has no attribute 'callbacks'

In [ ]:
# Exploring the future predictions


predictions_test.xs(246,level=1).tail()

## Notes on training time for the various algorithms:

In [ ]:
#These are calculated in minutes for the hh20 feature set (with about 40 features), for all 36 steps, calibration (c) and test (t) partitions, also include generating predictions, and are approximate:

#nj=12 (number of threads)
#scikit random forest:        21:13 (c), 26:20 (t) RandomForestRegressor(n_estimators=200, n_jobs=nj)
#XGB random forest:           06:02 (c), 07:51 (t) XGBRFRegressor(n_estimators=300,n_jobs=nj)
#scikit gbm:                  13:59 (c), 15:55 (t) GradientBoostingRegressor(), 
#scikit hurdle random forest: 07:32 (c), 09:49 (t) For both clf and reg: (n_estimators=200, n_jobs=nj)
#XGB hurdle xgb:              01:26 (c), 01:32 (t) For both clf and reg:                n_estimators=200,tree_method='hist',n_jobs=nj)
#scikit histgbm:              01:17 (c), 01:20 (t) HistGradientBoostingRegressor(max_iter=200)
#XGB xgb:                     01:00 (c), 01:04 (t) XGBRegressor(n_estimators=200,tree_method='hist',n_jobs=nj)
#lightgbm gbm:                00:25 (c), --    (t) LGBMRegressor(n_estimators=100,num_threads=8)

# Various helper functions and tools....

In [ ]:
!conda list | grep views-forecasts

# Retrieving external forecasts

In [ ]:
# Retrieve David's Markov models
# To do: rewrite the model dictionary to the new, slimmer version.
DRList = []


model = {
    'modelname':   'fat_hh20_Markov_glm',
    'algorithm': [],
    'depvar': "ln_ged_sb_dep",
    'data_train':      'hh20',
    'queryset': 'hh_20_features',
}
DRList.append(model)

model = {
    'modelname':   'fat_hh20_Markov_rf',
    'algorithm': [],
    'depvar': "ln_ged_sb_dep",
    'data_train':      'hh20',
    'queryset': 'hh_20_features',
}

DRList.append(model)



In [ ]:
path = f'/Users/{os.getlogin()}/Dropbox (ViEWS)/ViEWS/Projects/PredictingFatalities/Predictions/cm/preds/'

DRList[0]['predictions_file_calib'] = path + 'vmm_glm_hh20_0125_alt_calib.csv'
DRList[0]['predictions_file_test'] = path + 'vmm_glm_hh20_0125_alt_test.csv'
DRList[0]['predictions_file_future'] = path + 'vmm_glm_hh20_506.csv'

DRList[1]['predictions_file_calib'] = path + 'vmm_rf_hh20_0125_alt_calib.csv'
DRList[1]['predictions_file_test'] = path + 'vmm_rf_hh20_0125_alt_test.csv'
DRList[1]['predictions_file_future'] = path + 'vmm_rf_hh20_505.csv'

In [ ]:
print(path)

In [ ]:

for model in ModelList:
    print(model['modelname'])

In [ ]:
# Storing Markov models in central storage
# Retrieving dependent variable
target_calib = pd.DataFrame.forecasts.read_store('cm_fat_conflicthistory_rf_calib', run=run_id)['ln_ged_sb_dep']
target_test = pd.DataFrame.forecasts.read_store('cm_fat_conflicthistory_rf_test', run=run_id)['ln_ged_sb_dep']
level = 'cm'
for model in DRList:
    df_calib = pd.read_csv(model['predictions_file_calib'],index_col=['month_id','country_id'])
    df_test = pd.read_csv(model['predictions_file_test'],index_col=['month_id','country_id'])
    df_future = pd.read_csv(model['predictions_file_future'],index_col=['month_id','country_id'])
    df_calib['ln_ged_sb_dep'] = target_calib
    df_test['ln_ged_sb_dep'] = target_test
    df_future['ln_ged_sb_dep'] = np.nan # Empty dependent variable column for consistency/required by prediction storage function
    stored_modelname = level + '_' + model['modelname'] + '_calib'
    df_calib.forecasts.set_run(run_id)
    df_calib.forecasts.to_store(name=stored_modelname, overwrite=True)
    stored_modelname = level + '_' + model['modelname'] + '_test'
    df_test.forecasts.set_run(run_id)
    df_test.forecasts.to_store(name=stored_modelname, overwrite=True)    

In [ ]:
ModelList